# EzyPredict - prepare protein data

### Predicting enzyme function from protein sequence, structure and physicochemical properties


In [1]:
# Install dependencies
#!pip install cobra
#!pip install requests
#!pip install fair-esm
#!pip install torch
#!pip install tqdm

In [2]:
import cobra
import requests
import torch
from EzyPredict_helperfunctions import *
from cobra import io
import tqdm
import esm
import pickle
import subprocess
import re
import esm

## Define a set of enzymes

I don't want to predict the function of all types of proteins - just of enzymes. So, I need to select just enzymes from whole genomes. To start with, because I only have a local computer to train my model on, I will just extract all enzymes that are part of the Sachharomyces cerevisiae metabolic model.

I will do this using the latest genome-scale metabolic model of yeast - Yeast8.0 (https://github.com/SysBioChalmers/yeast-GEM) using the cobratoolbox (https://opencobra.github.io/cobratoolbox/stable/tutorials/) implemented in python as cobrapy (https://github.com/opencobra/cobrapy).


In [3]:

model_dir = "yeast-GEM/model"
model_file = next(f for f in os.listdir(model_dir) if f.endswith('.xml'))  # find the first .xml file in the directory
model_path = os.path.join(model_dir, model_file)

# Load the model
model = io.read_sbml_model(model_path)

enzymes = []

for reaction in model.reactions:
    # Get the list of enzymes (gene products) catalyzing the reaction
    enzymes.extend([gene.id for gene in reaction.genes])

# Remove duplicates by converting the list to a set and then back to a list
enzymes = list(set(enzymes))
#print(enzyme[1:5])

## Get sequences from UniProt

Uniprot doesn't take ORF Ids that are stored in 'enzymes' we need to convert these to UniProt identifiers. I will do this using a mapping file that was downloaded from UniProt.


In [4]:
## Convert ORF to UniProt
uniprot_ids = [convert_ORF2Uniprot(enzyme) for enzyme in enzymes]
#print(uniprot_ids[1:5])

## ESM2 encodings

### Load the ESM2 model 

In [5]:
## this downloads the ESM1b language model which is a 7GB file. Don't run more than once
# provided as a .zip in the repo
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S() 
batch_converter = alphabet.get_batch_converter()


## Fetch the sequence of each uniprotID and encode it

Using helper functions in EzyPredict_helperfunctions.py, I will pass every UniProtID of the enzymes I collected first to UniProt API and get an amino acid sequence. Then, I will pass it to ESM2 to get a sequence encoding.

In [6]:
## ESM2 doesn't accept sequences above 1024aa.
# I don't know what to do about this yet so i'm just trimming the sequences for now 

def encode_sequence_with_ESM2_trim(sequence):
    # If sequence is too long, trim it from both sides
    max_sequence_length = 1024 
    was_trimmed = False
    while len(sequence) > max_sequence_length:
        was_trimmed = True
        excess_length = len(sequence) - max_sequence_length
        trim_amount = (excess_length - 1) // 2  
        sequence = sequence[trim_amount:-trim_amount]
        
    # Prepare the data for ESM2
    data = [(sequence, sequence)] 
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    
    # Get the representations.
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33])
        token_representations = results["representations"][33]
    
    avg_representation = token_representations.mean(1)
    
    return avg_representation, was_trimmed


In [7]:
## Example of what the encodings look like 
#encoded_representations = {}

#for uid in uniprot_ids[0:3]:
 #   sequence = fetch_sequence_from_uniprot(uid)
  #  if sequence:
   #     encoding = encode_sequence_with_ESM2_trim(sequence)
    #    encoded_representations[uid] = encoding
#print(encoded_representations)


In [8]:
def save_to_disk(data, filename="ESM2_encoded_representations.pkl"):
    with open(filename, "wb") as f: 
        pickle.dump(data, f)

encoded_representations = {}
trimmed_count = 0 

# Loop with progress bar
for uid in tqdm.tqdm(uniprot_ids):
    sequence = fetch_sequence_from_uniprot(uid)
    if sequence:
        encoding, was_trimmed = encode_sequence_with_ESM2_trim(sequence)
        if not was_trimmed: ## I'm only saving the untrimmed because PCA later on showed that this was messing up the data
            encoded_representations[uid] = encoding
        else:
            trimmed_count += 1

# 
save_to_disk(encoded_representations)

# Print the number of trimmed sequences
print(f"{trimmed_count} sequences were trimmed due to exceeding the model's limit.")


100%|██████████| 1163/1163 [21:07<00:00,  1.09s/it]

70 sequences were trimmed due to exceeding the model's limit.


In [9]:
# Count how many sequences were trimmed
print(f"{trimmed_count} sequences were trimmed.")

70 sequences were trimmed.


In [10]:
# count how many sequences we have encoded
with open("ESM2_encoded_representations.pkl", "rb") as f:
    encoded_representations = pickle.load(f)
print(len(encoded_representations))


1093


## Pepstats

In [11]:
current_directory = os.getcwd()
directory = "pepstats_results"
import os
if not os.path.exists(directory):
    os.makedirs(directory)

### loop over all uniprot IDs, fetch sequence from UniProt API, then get pepstats from PEPSTATS API

## no need to run this again - all results are in pepstats_results
## Turn this into a Code cell if you want to run the fetching from pepstats api otherwise load the results 

for uid in tqdm.tqdm(uniprot_ids):
    
    # fetch sequence from UniProt
    sequence = fetch_sequence_from_uniprot(uid)
    
    # ensure sequence was successfully fetched
    if not sequence:
        print(f"Failed to fetch sequence for {uid}")
        continue
    
    # Submit the sequence to pepstats
    job_id = submit_sequence(sequence)
    
    # Keep checking every 30 seconds if the job is done
    while True:
        if is_job_done(job_id):
            break
        time.sleep(30)
    
    # Download results once the job is done
    filename = f"{uid}_pepstats"
    output_path = os.path.join(directory, filename)
    retrieve_job_results(job_id, output_path)

In [12]:
# After processing all sequences, collect results into a dataframe
pepstats_df = collate_pepstats_to_dataframe(directory)

# drop the columns that are not relevant for function and create big separation in PCA
pepstats_df = pepstats_df.drop(columns=["Molecular weight", "Average Residue Weight",
                                       "Improbability of expression in inclusion bodies",
                                       "A280 Molar Extinction Coefficients"])

#pepstats_df
pepstats_df

,Isoelectric Point,Tiny,Small,Aliphatic,Aromatic,Non-polar,Polar,Charged,Basic,Acidic,Total Residues
P48813,7.7144,202,331,211,87,397,266,134,73,61,663
P53241,9.0584,161,274,171,113,348,245,127,76,51,593
P33550,5.5352,95,179,94,79,220,205,115,57,58,425
Q02207,9.4507,271,469,283,102,496,404,213,123,90,900
P21375,9.4952,145,261,149,42,250,251,124,72,52,501
...,...,...,...,...,...,...,...,...,...,...,...
P38827,9.3539,275,492,253,123,466,614,351,204,147,1080
P53871,7.2464,103,179,99,43,182,175,98,54,44,357
P0CH64,5.9257,66,123,71,27,121,104,51,26,25,225
P32622,5.5463,135,254,138,50,243,252,135,67,68,495


#### Calculate fraction of Tiny, Small, Aliphate etc amino acids.. the total number is not important so after calculating the fraction , we remove it 

In [13]:
##
columns_to_divide = ["Tiny", "Small", "Aliphatic", "Aromatic", "Non-polar", "Polar", "Charged", "Basic", "Acidic"]

for column in columns_to_divide:
    pepstats_df[column] = pepstats_df[column].astype(float) / pepstats_df["Total Residues"]
pepstats_df = pepstats_df.drop('Total Residues', axis=1)


In [14]:
pepstats_df

,Isoelectric Point,Tiny,Small,Aliphatic,Aromatic,Non-polar,Polar,Charged,Basic,Acidic
P48813,7.7144,0.304676,0.499246,0.318250,0.131222,0.598793,0.401207,0.202112,0.110106,0.092006
P53241,9.0584,0.271501,0.462057,0.288364,0.190556,0.586847,0.413153,0.214165,0.128162,0.086003
P33550,5.5352,0.223529,0.421176,0.221176,0.185882,0.517647,0.482353,0.270588,0.134118,0.136471
Q02207,9.4507,0.301111,0.521111,0.314444,0.113333,0.551111,0.448889,0.236667,0.136667,0.100000
P21375,9.4952,0.289421,0.520958,0.297405,0.083832,0.499002,0.500998,0.247505,0.143713,0.103792
...,...,...,...,...,...,...,...,...,...,...
P38827,9.3539,0.254630,0.455556,0.234259,0.113889,0.431481,0.568519,0.325000,0.188889,0.136111
P53871,7.2464,0.288515,0.501401,0.277311,0.120448,0.509804,0.490196,0.274510,0.151261,0.123249
P0CH64,5.9257,0.293333,0.546667,0.315556,0.120000,0.537778,0.462222,0.226667,0.115556,0.111111
P32622,5.5463,0.272727,0.513131,0.278788,0.101010,0.490909,0.509091,0.272727,0.135354,0.137374


#### Normalise the Isoelectric point with standard scalar.. the others are already between 0 and 1

In [15]:
from sklearn.preprocessing import StandardScaler

cols_to_standardise = [
    'Isoelectric Point'
]

scaler = StandardScaler()

pepstats_normalised_df = pepstats_df.copy()

pepstats_normalised_df[cols_to_standardise] = scaler.fit_transform(pepstats_normalised_df[cols_to_standardise])
pepstats_normalised_df.to_csv("pepstats_results_standardised.csv")

## Create the labels - known EC numbers of each UniProt ID

In [16]:
ECnumbers_df = pd.read_csv('./databases/uniprot2EC_uniprotkb_download_2023_09_07.tsv', sep='\t')
ECnumbers_df.head()

,Entry,EC number
0,A0A0B7P3V8,2.7.7.49; 2.7.7.7; 3.1.26.4; 3.4.23.-
1,D6VTK4,NaN
2,O13297,3.6.1.74
3,O13329,NaN
4,O13516,NaN


In [17]:
ECnumbers_df = ECnumbers_df.assign(**{'EC number': ECnumbers_df['EC number'].str.split(';')}).explode('EC number').reset_index(drop=True)
ECnumbers_df['EC number'] = ECnumbers_df['EC number'].str.strip()

# Step 3: Discard everything beyond the second dot in the EC number
ECnumbers_df['EC number'] = ECnumbers_df['EC number'].str.extract('(\d+\.\d+)\.')

ECnumbers_df =ECnumbers_df.dropna(subset=['EC number'])

# Display the first few rows of the modified dataframe
print(ECnumbers_df.head())

        Entry EC number
0  A0A0B7P3V8       2.7
1  A0A0B7P3V8       2.7
2  A0A0B7P3V8       3.1
3  A0A0B7P3V8       3.4
5      O13297       3.6


## Combine all input variables and output class labels (EC numbers)

In [18]:
import pandas as pd
import pickle

# Load the sequence encodings
with open("ESM2_encoded_representations.pkl", "rb") as f:
    encoded_representations = pickle.load(f)

# Prepare data for DataFrame
data = [(key,) + tuple(value.numpy()[0]) for key, value in encoded_representations.items()]

# Prepare columns
columns = ['Entry'] + [f'Encoding_{i+1}' for i in range(1280)]

# Create dataframe
ESMencoding_df = pd.DataFrame(data, columns=columns)
ESMencoding_df.shape

(1093, 1281)

In [19]:

# Merge all three datasets based on the UniProtID
combined_df = pepstats_normalised_df.merge(ESMencoding_df, left_index=True, right_on='Entry')
combined_df = combined_df.merge(ECnumbers_df, on='Entry', how='inner')

# Retain only those rows which have non-missing values in all columns
combined_df = combined_df.dropna(how='any')

# Rearrange such that the Entry column is the first, and it's called "UniprotID"
combined_df.rename(columns={'Entry': 'UniprotID'}, inplace=True)
cols = ['UniprotID'] + [col for col in combined_df.columns if col != 'UniprotID' and col != 'EC number'] + ['EC number']
combined_df = combined_df[cols]

combined_df.head()


,UniprotID,Isoelectric Point,Tiny,Small,Aliphatic,Aromatic,Non-polar,Polar,Charged,Basic,...,Encoding_1272,Encoding_1273,Encoding_1274,Encoding_1275,Encoding_1276,Encoding_1277,Encoding_1278,Encoding_1279,Encoding_1280,EC number
0,P33550,-0.940575,0.223529,0.421176,0.221176,0.185882,0.517647,0.482353,0.270588,0.134118,...,0.006006,-0.078706,0.067504,-0.104536,0.024012,0.099171,-0.014483,0.063116,-0.103292,2.4
1,Q02207,1.390710,0.301111,0.521111,0.314444,0.113333,0.551111,0.448889,0.236667,0.136667,...,0.003671,-0.082104,0.138453,0.045217,0.080589,-0.004792,-0.045767,-0.085559,0.108033,1.1
2,Q02207,1.390710,0.301111,0.521111,0.314444,0.113333,0.551111,0.448889,0.236667,0.136667,...,0.003671,-0.082104,0.138453,0.045217,0.080589,-0.004792,-0.045767,-0.085559,0.108033,4.2
3,P21375,1.417205,0.289421,0.520958,0.297405,0.083832,0.499002,0.500998,0.247505,0.143713,...,0.049613,-0.072365,0.065138,-0.225438,0.022500,-0.035332,-0.035813,0.007896,0.042926,1.3
4,P04046,-0.546599,0.284314,0.523529,0.290196,0.107843,0.547059,0.452941,0.268627,0.137255,...,-0.129664,-0.186735,0.083157,-0.123854,-0.139891,0.068116,-0.059735,-0.066775,-0.084309,2.4


In [20]:
combined_df.to_csv("all_prepareddata_for_EzyPredict.csv", index = False)

In [21]:
combined_df.shape

(944, 1292)